# Slimming down cleaning process and make it able to take in any new data
##### Started 14.12.22

### Imports

In [3]:
import pandas as pd
import random
import string
import re
import json
import ast
from collections import Counter


### Getting workable DF

In [4]:
data_path = '~/code/emilycardwell/final-project-data/data/raw/kaggle_raw.csv'
raw_df = pd.read_csv(data_path)
raw_df.head(3)

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,{3: 'G G/B ...,"{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,{1: 'Em D C C...,"{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber


In [5]:
slim_raw_df = raw_df[['artist_name', 'song_name', 'chords', 'genres', 'popularity']]
first_raw = ast.literal_eval(slim_raw_df.loc[0, 'chords'])
first_raw

{3: 'G             G/B                         C ',
 5: '                  G ',
 7: 'G                  G/B                         C ',
 9: '                G ',
 13: '         G         Em             C       G     ',
 15: '     G        Em                C   G ',
 19: '          G            Em        C            G    ',
 21: '      G              Em                  C              G    ',
 23: '      G           Em             C         G    ',
 25: '        G            Em           C          G    ',
 27: '           G        Em       C        G            ',
 31: 'G             G/B                       C ',
 33: '                 G ',
 35: 'G                G/B                C        ',
 37: '                G ',
 41: '         G         Em             C       G     ',
 43: '     G        Em                C   G ',
 47: '          G            Em        C            G    ',
 49: '      G              Em                  C              G    ',
 51: '      G           Em       

In [ ]:
    # def sub_spaces_for_reps(measure):
    #     hole = int(max(map(len, re.findall(r'\s+', measure))))
        
    #     new = r'[^\s]+(?=\s{2,})'
        
    #     dash_rep = re.sub(r'\s{2,}', ' - ', measure)
    #     reps_list = re.findall(new, measure)
        
    #     d = dash_rep
    #     for x in reps_list:
    #         d = re.sub('- -', x, d, 1)
    #         d = re.sub('-', x, d, 1)
        
    #     return d

In [227]:
def string_cleaning(df):
    split_raw_df = df.copy()
    splits = []
        
    for row in split_raw_df.chords:
        row_dict = ast.literal_eval(row)
        all_chords = []
        
        for mms, cc in row_dict.items():
            all_chords.append([mms, cc])
        
        for i, chords in enumerate(all_chords):
            if re.findall(r'\S+(?=\s*(%|2x|x2|\s-\s|))', chords[1]):
                reps = re.findall(r'\S+(?=\s*(%|2x|x2))', chords[1])
                for x in reps:
                    rep = re.sub(r'%|2x|x2', x, chords[1], 1)
            else:
                rep = chords[1]
            
            p1 = re.sub(r'-+|[()|,.]', ' ', rep)
            p2 = p1.replace('\\', '')
            p = re.sub(r'\s+$', '', p2)            
            
            if re.match(r'^\s{3,}', p) and i != 0:
                last = re.findall(r'(\S+$)', all_chords[i-1][1])[0]
                try:
                    s = re.sub(r'^\s+', f'{last} ', p)
                except re.error:
                    print(all_chords[i-1][1], ':', last)
                r = re.sub(r'\s+', ' ', s)
                all_chords[i][1] = r
            else:
                s = re.sub(r'^\s+', '', p)
                r = re.sub(r'\s+', ' ', s)
                all_chords[i][1] = r
                
        splits.append(all_chords)
        
    split_raw_df.chords = splits
    
    return split_raw_df

In [222]:
# try this
split_raw_df = string_cleaning(slim_raw_df)
split_raw_df.chords

0         {3: 'G             G/B                        ...
1         {1: 'Intro: F#m7  D2 ', 4: 'F#m7 ', 8: 'D2 ', ...
2         {1: 'Em            D         C               C...
3         {4: ' Intro: Em  Bm  Am  C   (2x) ', 6: '     ...
4         {2: 'Intro: Gm - Dm - C - C   x2 ', 5: '      ...
                                ...                        
135778    {0: '
	  		B                            E ', 2...
135779    {2: 'Intro: A5(9) ', 5: 'A5(9)                ...
135780    {2: 'Dm                     C   ', 4: '       ...
135781    {3: 'G             Em ', 5: '    C        G ',...
135782    {0: '
	  		(Intro:) D F G F   D F G F   D F G ...
Name: chords, Length: 135783, dtype: object

In [212]:
r = random.randint(0, 135782)
print(slim_raw_df.chords[r])
print(split_raw_df.chords[r])

{1: 'Intro: E A B A E ', 3: 'E                                         A ', 5: 'B ', 7: 'A                            E ', 9: 'E                                          A ', 11: 'B ', 13: '         A                      E ', 16: 'C#m                                       A ', 18: 'B                             E ', 20: 'C#m                              A ', 22: 'B ', 24: '             A                      E ', 27: 'Solo ', 29: 'E                                            A ', 31: 'B ', 33: '          A                 E ', 36: 'E                                             A ', 38: '      B ', 40: '              A                     E '}
{1: 'Intro: E A B A E ', 3: 'E                                         A ', 5: 'B ', 7: 'A                            E ', 9: 'E                                          A ', 11: 'B ', 13: '         A                      E ', 16: 'C#m                                       A ', 18: 'B                             E ', 20: 'C#m                     

#### See Chord Frequency

In [215]:
chords_counter = Counter()
for song in split_raw_df.chords:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)
del chords_counter['']

chord_count_df = pd.Series(dict(chords_counter)).to_frame('chord_count')
sorted_cc_df = chord_count_df.sort_values(by='chord_count', ascending=False)
len(sorted_cc_df)

103294

In [216]:
sorted_cc_df.head(100)

,chord_count
G,1029337
C,846337
D,794185
A,631590
F,511927
...,...
Gdim,7275
F5,7272
G#7,6968
Dsus2,6895


In [ ]:
letters = list(string.ascii_uppercase)[:7]
non_chord_filter = [chord for chord in sorted_cc_df.index if chord[0] in letters]
cc_df = sorted_cc_df[sorted_cc_df.index.isin(non_chord_filter)]

# pd.set_option('display.max_rows', 1190)
## sorted_cc_df.head(70*17)
pd.set_option('display.max_rows', 100)
cc_df.iloc[100:200]

In [ ]:
# slim_chord_counts_dict = {}
# for chord, count in chords_count_dict.items():
#     if count <= low_freq_to_remove:
#         pass
#     else:
#         slim_chord_counts_dict[chord] = count

## Creating List of Correctly Formatted Chords

In [ ]:
major_chords = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']

In [ ]:
chords_df = pd.DataFrame(index=['', 'm', 'dim', 'aug', '5', 'aug5', '5b7',
                                'dim7', 'hdim7', 'm7', 'm-M7', '7', 'aug7', 'M7',  
                                'm9', '9', '7b9',  'M9', 'm11', '11',
                                'add2', 'add4', 'add6', 'add9', 
                                'madd2', 'madd4', 'madd6', 'madd9', 
                                '7add4', '7addb6', '7add6', 
                                'msus2', 'msus4', '7sus2', '7sus4', 'sus2', 'sus4'
                                ],
                         columns=major_chords)


In [ ]:
for idx in chords_df.index:
    new_chords = [m + idx for m in major_chords]
    chords_df.loc[idx, :] = new_chords
chords_df.rename(index={'': 'major'}, inplace=True)
chords_df.head(3)

In [ ]:
sharp_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 
                'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
flats_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 
                'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
sharp_keys = ['D', 'E', 'G', 'A', 'B', 'Dm', 'Em', 'Fm', 'Bm']
flats_keys = ['C', 'F', 'Cm', 'Gm', 'Am']

slash_indexes = ['dim/b7', 'm/2', 'm/b3', 'm/3', 'm/4', 'm/5', 'm/6','m/b7', 'm/7',
                'm7/b3', 'm7/4', 'm7/b5', 'm7/5', 'm7/7', 
                '/2', '/b3', '/3', '/4', '/5', '/6','/b7', '/7', 
                '7/b2', '7/2', '7/b3', '7/3', '7/4', '7/5', '7/b6', '7/6','7/7', 
                '9/b3', '9/3', '9/4', '9/5', '9/6', '9/7',
                'M7/2', 'M7/b3', 'M7/3', 'M7/4', 'M7/5', 'M7/6','M7/b7']

whole_to_half = {'b2': 1, '2': 2, 'b3': 3, '3': 4, '4': 5, 'b5': 6,
                 '5': 7, 'b6': 8,'6': 9, 'b7': 10, '7': 11}

In [ ]:
# big function
def get_slash_notes(slash_indexes, columns):
    slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
    i = 0 
    
    def find_slash(chord, notes, i):
        for si in slash_indexes:
            s = si.split('/')
            h = whole_to_half[s[1]]
            sc = f'{chord}{s[0]}/{notes[i+h]}'
            slash_chord.loc[si, chord] = sc
    
    def sort_f_s(chord, notes):
        for n in notes:
            if len(chord) > 1:
                i = notes.index(chord)
                find_slash(chord, notes, i)
            else:
                i = notes.index(chord[0])
                find_slash(chord, notes, i)
    
    for chord in columns:
        if chord in sharp_keys or '#' in chord:
            sort_f_s(chord, sharp_notes)
        else:
            sort_f_s(chord, flats_notes)
                
            
    return slash_chord

In [ ]:
slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
slash_chords_df

## Shove chords into correct format

In [ ]:
substitutions = {'maj': '', ',': '', 'º': 'dim', '*': 'dim', '5+': 'aug', '7M': 'M7',
                 'maj7': 'M7', 'm7b5': 'hdim7', 'm7+': 'm-M7', '7+': 'aug7', 
                 '79': '9', 'm5-/7': 'dim/b7', '711': '7add4', '7b13': '7addb6', 
                 '713': '7add6', '4/7/9': '9add4', '69': '9add6', '/9-': '/b2', 
                 '/9': '/2', '/5-': '/b5', '/13-': '/b6', '/5+': '/b6', '/13': '/6',
                 '2': 'add2', '4': 'add4', '6': 'add6', 'sus': 'sus4'
                 }

In [ ]:
def merge_chords(chords_column):
    chords_column_copy = chords_column.copy()
    merged_chords = []
    rejected = []
    added = []
    
    # compare against df
    def counters(chord):
        if chord in slash_chords_df.columns:
            added.append(chord)
        elif chord in slash_chords_df.values:
            print(chord, 'in_df')
            added.append(chord)
        else:
            print(chord, 'womp')
            rejected.append(f'{chord}-f')
        return 
    
    # if keys in subs in chord, replace with values
    def check_subs(chord):
        for k, s in substitutions.items():
            if k in chord: 
                print(k, chord)
                ch_sub = chord.replace(k, s)
                print(ch_sub, 'subbed')
                return ch_sub
            else:
                return chord
    
    for song in chords_column_copy:
        merged_song = {}
        for m, chords in song.items():
            measure = chords.split()
            for idx, ch in enumerate(measure):
                            
                            # START HERE

                if idx == 0 and m == ?:
                    chs = check_subs(ch)
                    counters(chs)
                    measure[idx] = chs
                else:
                    chs = check_subs(ch)
                    counters(chs)
                    measure[idx] = chs                
            merged_song[m] = measure
        merged_chords.append(merged_song)
    
    added_c = Counter(added)
    rejects_c = Counter(rejected)
    
    added_df = pd.Series(dict(added_c)).to_frame('added').sort_values(by='added', ascending=False)
    rejects_df = pd.Series(dict(rejects_c)).to_frame('rejects').sort_values(by='rejects', ascending=False)
 
    return pd.DataFrame(merged_chords).T, added_df, rejects_df

In [ ]:
# merged, rejects, adds = merge_chords(split_raw_df.chords[0])
split_raw_df.chords[0]

#### For next time:
- fix rejects (reject) tag

In [ ]:
rejects_counter = Counter()
for r in rejects:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)

rejects_df = pd.Series(dict(rejects_counter)).to_frame('rejects')
sorted_rej_df = rejects_df.sort_values(by='rejects', ascending=False)
sorted_rej_df

In [ ]:
# # big function
# def translate_slashes(slash_indexes, columns):
#     slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
#     i = 0 
    
#     def find_slash(chord, notes, i):
#         for si in slash_indexes:
#             s = si.split('/')
#             h = whole_to_half[s[1]]
#             sc = f'{chord}{s[0]}/{notes[i+h]}'
#             slash_chord.loc[si, chord] = sc
    
#     def sort_f_s(chord, notes):
#         for n in notes:
#             if n in chord :
#                 i = notes.index(chord[0])
#                 find_slash(chord, notes, i)
#             elif len(chord) > 1:
#                 if '#' == chord[1]:
#                     i = sharp_notes.index(chord[:2])
#                     find_slash(chord, sharp_notes, i)
#                 elif 'b' == chord[1]:
#                     i = flats_notes.index(chord[:2])
#                     find_slash(chord, flats_notes, i)
    
#     for chord in columns:
#         sort_f_s(chord, sharp_notes)
#         sort_f_s(chord, flats_notes)
                
            
#     return slash_chord

In [ ]:
# slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
